all the imports

In [1]:
import arxiv
import requests
import json

In [2]:
def get_arxiv_papers(query: str, max_papers: int = 5) -> list[dict]:
    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results = max_papers)
    papers = []

    for result in client.results(search):
        paper = {
            "title" : result.title,
            "authors": [result.author.name],
            "summary": result.summary,
            "published": result.published.date(),
            "url": result.pdf_url
        }
        papers.append(paper)

    return papers

In [6]:
from tenacity import retry, stop_after_attempt

@retry(stop=stop_after_attempt(5))


def get_ieee_papers(query:str, api_key:str, max_results:int = 5) -> list[dict]:
    url = "https://ieeexploreapi.ieee.org/api/v1/search/articles?parameter&apikey="
    params = {
        "query": query,
        'apikey': api_key,
        'max_records': max_results,
        'format': 'json'
    }

    response = requests.get(url, params=params, timeout=10)
    if response.status_code != 200:
        raise Exception(f'IEEE Xplore API failed: {response.text}')
    
    papers = []

    for article in response.json().get('articles', []):
        papers.append({
            "title": article.get('title', 'no title'),
            "authors": [author["full_name"] for author in article.get('authors', [])],
            "abstract":article.get('abstract', 'No abstract'),
            'published': article.get('publication_date',""),
            'url': article.get('pdf_url', article.get('html_url',''))
        })
    return papers

    

In [8]:
def research_agent(query:str, ieee_api_key:str, max_papers: int = 5)-> list[str]:
    try:
        papers = get_arxiv_papers(query, max_papers)
        if not papers:
            return ValueError('arXiv returned no papers')
        return papers
    except Exception as e:
        if not ieee_api_key:
            return ValueError('Switching to IEEE - Need the API key')
        print(f'arxiv failed ({e}), trying IEEE Xplore')
        return get_ieee_papers(query, ieee_api_key, max_papers)



In [9]:
def get_results(papers: list[dict]):
    for i, paper in enumerate(papers,1):
        print(f"""
        [{i}] {paper['title']}
        Authors:{','.join(paper.get('authors', []))}
        Published: {paper.get('published', 'N/A')}
        Summary: {paper.get('summary', paper.get('abstract', 'N/A'))[:200]}...
        URL:{paper.get('url','N/A')}
        """)